In [1]:
import pandas as pd
import requests
import json

In [2]:
def req(obj):
    url = 'https://ring.org.ua/search?q=' + obj + '&datasources=edrdr&format=json'
    response = requests.get(url) # надсилаємо get-запит на сервер ring.org.ua
    #response = requests.get(url, verify=False) # якщо матюкається на SSL-сертифікат
    if response.status_code == 200:
        dict_resp = response.json()
        items = dict_resp['search_results']['paginator']['count'] # кількість пов'язаних фірм
        pages = dict_resp['search_results']['paginator']['num_pages'] # кількість сторінок у результаті
        result = dict_resp['search_results']['object_list'] # формуємо список даних про фірми
        if pages > 1: # якщо результати запиту не вмістилися на одну сторінку (не може бути більше 1000?)
            for i in range(2, pages+1): # повторюємо запит для всіх сторінок, починаючи з другої
                url = 'https://ring.org.ua/search?q=' + obj + '&datasources=edrdr&format=json&page=' + str(i)
                response = requests.get(url) # надсилаємо get-запит на сервер ring.org.ua
                #response = requests.get(url, verify=False) # якщо матюкається на SSL-сертифікат
                if response.status_code == 200:
                    dict_resp = response.json()
                    result += dict_resp['search_results']['object_list'] # поповнюємо список даних про фірми
                else:
                    print('Error: requested url ' + url + ' is not available!')
        if len(result) == items: # перевіряємо, чи нам вдалося отримати інформацію про всі пов'язані фірми
            return (items, result)
        else:
            print('Warning: parsed url ' + url.split('&page=')[0] + ' is not full!')
            return (len(result), result)
    else:
        print('Error: requested url ' + url + ' is not available!')
        return (0, ['error'])

In [3]:
def add_info(firm):
    # додаємо фірму до словника знайдених контрагентів
    edrpou = firm['full_edrpou']
    name = firm['latest_record']['name'].lower()
    short_name = firm['latest_record']['short_name'].lower()
    status = firm['latest_record']['status'].lower()
    dict_find[edrpou] = (name, short_name, status) # додаємо інформацію про фірму: назва, повна назва, статус
    dict_firm[name] = edrpou # якщо будуть дублі за повною назвою фірми, то запишеться лише останній ЄДРПОУ!
    if (status != 'припинено') and (name not in list_temp) and (name not in list_next): # якщо фірма функціонує
        list_next.append(name) # на наступному рівні шукатимемо засновані цією фірмою фірми
        # додаємо інформацію про засновників фірми
        list_all = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons']])) # фізособи
        for person in list_all:
            if person not in set(dict_find.keys()): # якщо ця особа ще не засвітилася раніше
                dict_find[person] = () # додаємо її П.І.Б. до словника знайдених контрагентів
            if (person not in list_temp) and (person not in list_next): # і якщо вона не включена до пошуку
                list_next.append(person) # на наступному рівні шукатимемо засновані цією особою фірми
        list_master = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons'] 
                                if person[1] == 'Голова'])) # список голів фірми (голова тільки один?)
        list_founder = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons'] 
                                 if person[1] == 'Засновник'])) # список засновників (вносили кошти)
        list_benef = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons'] 
                               if person[1] == 'Бенефіціарний власник'])) # список бенефіціарів
        for head in list_master: # додаємо інформацію про голів фірм, вони можуть не бути засновниками
            list_head.append([head, edrpou])
        records = firm['raw_records'] # тут міститься інформація про частки участі засновників (у грн.)
        part = dict()
        for record in records: # відбираємо інформацію тільки про ненульові внески
            if (record.find(' грн.')!=-1) and (record.find(' 0.00 грн.')==-1) and (record.find(' 0,00 грн.')==-1):
                part[record.split(',')[0].lower().replace('  ', ' ')] = float(
                    record.split('-')[-1].split('грн')[0].strip().replace(',', '.')) # засновник і його внесок
        if len(part) > 0: # якщо є інформація хоча б про один ненульовий внесок до статутного капіталу
            capital = sum(part.values())
            for founder in list(part):
                if founder in list_all: # засновник -- фізична особа
                    if founder in list_benef: # засновник є бенефіціаром
                        list_type.append([founder, edrpou, part[founder]/capital, 1])
                    else: # засновник не є бенефіціаром
                        list_type.append([founder, edrpou, part[founder]/capital, 0])
                else: # засновник -- юридична особа (фірма)
                    if founder in list(dict_firm): # і для цієї фірми ми вже маємо інформації про її ЄДРПОУ
                        list_type.append([dict_firm[founder], edrpou, part[founder]/capital, 0])
                    else: # інакше записуємо просто назву фірми
                        list_type.append([founder, edrpou, part[founder]/capital, 0])
        # якщо бенефіціар не вносив кошти до статутного капіталу, додаємо інформацію про нього
        zero_benef = list(set(list_benef) - set(list(part)))
        if len(zero_benef) > 0:
            for founder in zero_benef:
                list_type.append([founder, edrpou, 0, 1])
        # якщо засновник не є бенефіціаром і не вносив кошти до статутного капіталу, додаємо інформацію про нього
        zero_founder = list(set(list_founder) - set(list_benef) - set(list(part)))
        if len(zero_founder) > 0:
            for founder in zero_founder:
                list_type.append([founder, edrpou, 0, 0])
        #print('Всі засновники: ', list_all)
        #print('Голови: ', list_master)
        #print('Звичайні засновники: ', list_founder)
        #print('Засновники-бенефіціари: ', list_benef)
        #print('Засновники з ненульовими внесками: ', list(part))
        #print('"Нульові" засновники-бенефіціари: ', zero_benef)
        #print('"Нульові" звичайні засновники: ', zero_founder)

In [15]:
dict_find = dict() # словник знайдених контрагентів
dict_firm = dict() # словник назва фірми --> ЄДРПОУ
list_type = [] # список зв'язків між знайденими контрагентами
list_head = [] # список голів фірм
list_temp = [] # список контрагентів на поточному рівні
list_next = [] # список контрагентів наступного рівня
i = 0 # лічильник (номер рівня відносно початкового об'єкта)

In [16]:
# ці дані буде задавати користувач під час запуску
start = '35696771' # задаємо ЄДРПОУ початкової фірми
level = 3 # задаємо максимальну кількість рівнів

In [17]:
%%time
list_next.append(start)
while (len(list_next) > 0) and (i < level):
    list_temp = list_next.copy()
    list_next = []
    for obj in list_temp:
        resp = req(obj)
        num = resp[0] # кількість знайдених фірм, пов'язаних із даним об'єктом (ЄДРПОУ, назва фірми, П.І.Б.)
        if num != 0:
            list_obj = resp[1] # список даних по знайдених фірмах
            for n in range(num):
                edrpou = list_obj[n]['full_edrpou'] # підтягуємо ЄДРПОУ фірми (що робити, якщо його немає?)
                if edrpou not in set(dict_find.keys()): # якщо ми ще не обробляли інформацію про дану фірму
                    add_info(list_obj[n]) # додаємо інформацію про засновників фірми та нові об'єкти для пошуку
        else:
            if req(obj)[1] == ['error']:
                print('Error request for object = ', obj)
    i += 1

CPU times: user 2.66 s, sys: 105 ms, total: 2.76 s
Wall time: 36.9 s


In [18]:
dict_find # знайдені контрагенти (вершини графа), для фірм додаємо повну назву, скорочену назву і стан

{'35696771': ('товариство з обмеженою відповідальністю "комплекс інвестмент менеджмент"',
  'тов "кім"',
  'зареєстровано'),
 'ловчин лев юрійович': (),
 'соколенко олександр іванович': (),
 'рутовський дмитро олегович': (),
 '37761449': ('товариство з обмеженою відповідальністю "інджи"',
  'тов "інджи"',
  'зареєстровано'),
 'меламуд євген мойсейович': (),
 'кравченко валентин валерійович': (),
 '38663859': ('приватне підприємство "щедрий сад."',
  'пп "щедрий сад."',
  'зареєстровано'),
 '38141129': ('фермерське господарство "соколенки"',
  'фг "соколенки"',
  'зареєстровано'),
 'соколенко сергій іванович': (),
 '21503248': ('товариство з обмеженою відповідальністю "фірма "сезон лтд."',
  'тов "фірма "сезон лтд."',
  'зареєстровано'),
 'бернатович вадим миколайович': (),
 'ємельянова ольга олександрівна': (),
 'соколенко іван станіславович': (),
 '13904523': ('мале підприємство у вигляді товариства з обмеженою відповідальністю фірма "сокол" лтд',
  'мп тов "сокол" лтд',
  'припинено'

In [19]:
dict_firm # словник для визначення ЄДРПОУ фірми за її назвою (сподіваюсь, дублів за повними назвами фірм немає)

{'товариство з обмеженою відповідальністю "комплекс інвестмент менеджмент"': '35696771',
 'товариство з обмеженою відповідальністю "інджи"': '37761449',
 'приватне підприємство "щедрий сад."': '38663859',
 'фермерське господарство "соколенки"': '38141129',
 'товариство з обмеженою відповідальністю "фірма "сезон лтд."': '21503248',
 'мале підприємство у вигляді товариства з обмеженою відповідальністю фірма "сокол" лтд': '13904523',
 'товариство з обмеженою відповідальністю "камидана"': '40581613',
 'товариство з обмеженою відповідальністю "сервіс уют"': '39189177',
 'товариство з обмеженою відповідальністю "домера"': '39119103',
 'товариство з обмеженою відповідальністю "южспецмонтаж донбас"': '33852359',
 'товариство з обмеженою відповідальністю "южспецмонтаж"': '30542625',
 'дочірнє підприємство "юсм-уют"': '33138719',
 'товариство з обмеженою відповідальністю "стройбілдінг"': '39324857',
 'товариство з обмеженою відповідальністю "коляда софт"': '42498353',
 'товариство з обмеженою ві

In [20]:
list_type # (засновник, ЄДРПОУ заснованої фірми, частка участі засновника, чи є засновник бенефіціаром)

[['соколенко олександр іванович', '35696771', 0.34, 1],
 ['рутовський дмитро олегович', '35696771', 0.33, 0],
 ['ловчин лев юрійович', '35696771', 0.33, 0],
 ['меламуд євген мойсейович', '37761449', 0.45, 0],
 ['35696771', '37761449', 0.55, 0],
 ['соколенко олександр іванович', '38663859', 1.0, 0],
 ['соколенко олександр іванович', '38141129', 0, 0],
 ['бернатович вадим миколайович', '21503248', 0.02, 0],
 ['соколенко іван станіславович', '21503248', 0.04, 0],
 ['ємельянова ольга олександрівна', '21503248', 0.9, 0],
 ['соколенко олександр іванович', '21503248', 0.04, 0],
 ['рутовський дмитро олегович', '40581613', 0.2792307142272753, 0],
 ['рутовська світлана олександрівна', '40581613', 0.0837692142681826, 0],
 ['товариство з обмеженою відповідальністю "южспецмонтаж"',
  '40581613',
  0.6370000715045421,
  0],
 ['рутовський дмитро олегович', '39189177', 1.0, 1],
 ['товариство з обмеженою відповідальністю "южспецмонтаж"',
  '39119103',
  0.23026733809523808,
  0],
 ['рутовський дмитро о

Якщо ЄДРПОУ по якихось фірмах не підтягнулося відразу, то заміняємо зараз назву фірми на ЄДРПОУ (заміняється не завжди):

In [21]:
%%time
list_type_new = list_type.copy()
for i in range(len(list_type_new)):
    if list_type_new[i][0] in list(dict_firm):
        list_type_new[i][0] = dict_firm[list_type_new[i][0]]
list_type_new

CPU times: user 1.11 ms, sys: 95 µs, total: 1.2 ms
Wall time: 1.22 ms


[['соколенко олександр іванович', '35696771', 0.34, 1],
 ['рутовський дмитро олегович', '35696771', 0.33, 0],
 ['ловчин лев юрійович', '35696771', 0.33, 0],
 ['меламуд євген мойсейович', '37761449', 0.45, 0],
 ['35696771', '37761449', 0.55, 0],
 ['соколенко олександр іванович', '38663859', 1.0, 0],
 ['соколенко олександр іванович', '38141129', 0, 0],
 ['бернатович вадим миколайович', '21503248', 0.02, 0],
 ['соколенко іван станіславович', '21503248', 0.04, 0],
 ['ємельянова ольга олександрівна', '21503248', 0.9, 0],
 ['соколенко олександр іванович', '21503248', 0.04, 0],
 ['рутовський дмитро олегович', '40581613', 0.2792307142272753, 0],
 ['рутовська світлана олександрівна', '40581613', 0.0837692142681826, 0],
 ['30542625', '40581613', 0.6370000715045421, 0],
 ['рутовський дмитро олегович', '39189177', 1.0, 1],
 ['30542625', '39119103', 0.23026733809523808, 0],
 ['рутовський дмитро олегович', '39119103', 0.6983040904761905, 1],
 ['рутовський олег георгійович', '39119103', 0.03571428571

In [11]:
list_head # (голова, ЄДРПОУ фірми)

[['соколенко олександр іванович', '35696771'],
 ['кравченко валентин валерійович', '37761449'],
 ['соколенко олександр іванович', '38663859'],
 ['соколенко сергій іванович', '38141129'],
 ['соколенко іван станіславович', '21503248'],
 ['рутовський дмитро олегович', '40581613'],
 ['корольов сергій петрович', '39189177'],
 ['рутовський дмитро олегович', '39119103'],
 ['анісіфоров володимир львович', '30542625'],
 ['корольов петро васильович', '30542625'],
 ['галак оксана василівна', '30542625'],
 ['малюта ірина михайлівна', '30542625'],
 ['саіпова наталія леонідівна', '33138719'],
 ['жоров євген данилович', '39324857'],
 ['тростянецький володимир васильович', '42498353'],
 ['гук вячеслав олександрович', '42498353'],
 ['діланян самвел авакович', '41398320'],
 ['дорошенко анатолій іванович', '41398320'],
 ['тростянецький володимир васильович', '41398320'],
 ['анісіфоров володимир львович', '39009435'],
 ['корольов петро васильович', '39009435'],
 ['нарольський андрій олегович', '39009435']

In [12]:
list_temp # список контрагентів, за якими здійснювався пошук на останньому рівні

['товариство з обмеженою відповідальністю "інджи"',
 'меламуд євген мойсейович',
 'кравченко валентин валерійович',
 'приватне підприємство "щедрий сад."',
 'фермерське господарство "соколенки"',
 'соколенко сергій іванович',
 'товариство з обмеженою відповідальністю "фірма "сезон лтд."',
 'бернатович вадим миколайович',
 'ємельянова ольга олександрівна',
 'соколенко іван станіславович',
 'товариство з обмеженою відповідальністю "камидана"',
 'рутовська світлана олександрівна',
 'товариство з обмеженою відповідальністю "сервіс уют"',
 'корольов сергій петрович',
 'товариство з обмеженою відповідальністю "домера"',
 'рутовський олег георгійович',
 'товариство з обмеженою відповідальністю "южспецмонтаж"',
 'галак оксана василівна',
 'анісіфоров володимир львович',
 'малюта ірина михайлівна',
 'корольов петро васильович',
 'дочірнє підприємство "юсм-уют"',
 'саіпова наталія леонідівна',
 'товариство з обмеженою відповідальністю "стройбілдінг"',
 'жоров євген данилович',
 'товариство з обм

In [13]:
list_next # список контрагентів для пошуку на наступному рівні (непорожній, якщо було задано максимальний рівень)

['мале підприємство - товариство з обмеженою відповідальністю "онікс - лтд"',
 'зак вячеслав юхимович',
 'громадська організація "організація інвалідів "новий крок"',
 'стрельников андрій вікторович',
 'шейко ольга вікторівна',
 'приватне підприємство "щедрий сад"',
 'шеремет олег петрович',
 'виробничо - комерційна приватна фірма "тік"',
 'соколенко іван сергійович',
 'товариство з обмеженою відповідальністю виробниче підприємство "комплекс"',
 'швачко геннадій євгенович',
 'приватне підприємство "міжнародний центр іноваційних технологій, інформації, підприємництва, бізнесу, організації праці та навчання"',
 'коваленко володимир григорович',
 'руденко олег іванович',
 'бондаренко василь павлович',
 'товариство з обмеженою відповідальністю "фірма "фрізон лтд."',
 'майсак сергій григорович',
 'осетров валентин геннадійович',
 'донченко євгеній васильович',
 'майсак ігор григорович',
 'товариство з обмеженою відповідальністю "уют сервіс"',
 'олійник олег володимирович',
 'товариство з об